# FKM Nonlinear example
## Plot a load-lifetime woehler curve with several calls to the FKM nonlinear algorithm

#### Python module imports

In [ ]:
# standard modules
import pandas as pd
import numpy as np
import itertools
import timeit
import matplotlib.pyplot as plt
import matplotlib.patches

# pylife
import pylife
import pylife.strength
import pylife.strength.fkm_nonlinear
import pylife.mesh.gradient

## Collect all input data

In [ ]:
# define function for one FKM assessment
def get_lifetime(load):
    load_sequence = pd.Series([load, 0])  # [N]
    
    assessment_parameters = pd.Series({
            'MatGroupFKM': 'Steel',  # [Steel, SteelCast, Al_wrought] material group
            'R_m':  400,  #907,           # [MPa] ultimate tensile strength (de: Zugfestigkeit)
            #'K_RP': 1,             # [-] surface roughness factor, set to 1 for polished surfaces or determine from the diagrams below
            'R_z': 250,  #0,             # [um] average roughness (de: mittlere Rauheit), only required if K_RP is not specified directly

            'P_A': 0.5,            # [-] (one of [0.5, 2.3e-1, 1e-3, 7.2e-5, 1e-5], failure probability (de: auszulegende Ausfallwahrscheinlichkeit)
            # beta: 0.5,           # damage index, specify this as an alternative to P_A

            'P_L': 50,             # [%] (one of 2.5%, 50%) (de: Auftretenswahrscheinlilchkeit der Lastfolge)
            'c':   1.4,              # [MPa/N] (de: Übertragungsfaktor Vergleichsspannung zu Referenzlast im Nachweispunkt, c = sigma_I / L_REF)
            'A_sigma': 339.4,  # 25,         # [mm^2] (de: Hochbeanspruchte Oberfläche des Bauteils)
            'A_ref': 500,          # [mm^2] (de: Hochbeanspruchte Oberfläche eines Referenzvolumens), usually set to 500
            'G': 2/15, #0.44,             # [mm^-1] (de: bezogener Spannungsgradient)
            's_L': 10, # 0,              # [MPa] standard deviation of Gaussian distribution
            'K_p': 3.5, #2.76,           # [-] (de: Traglastformzahl) K_p = F_plastic / F_yield (3.1.1)
    })


    result = pylife.strength.fkm_nonlinear.assessment_nonlinear_standard.perform_fkm_nonlinear_assessment(assessment_parameters, load_sequence, 
                                                                            calculate_P_RAM=True, calculate_P_RAJ=True)

    print(result["P_RAM_stddev_log_N"], result["P_RAJ_stddev_log_N"])
    
    # get lifetime for P_RAM
    n_ram = result['P_RAM_lifetime_n_cycles']
    if result['P_RAM_is_life_infinite']:
        n_ram = 1e7
        
        
    # get lifetime for P_RAJ
    n_raj = result['P_RAJ_lifetime_n_cycles']
    if result['P_RAJ_is_life_infinite']:
        n_raj = 1e7
        
    # get damage parameter values
    p_ram = result['P_RAM_collective']["P_RAM"].values[0]
    p_raj = result['P_RAJ_collective']["P_RAJ"].values[0]
    
    # get scatter
    one_over_tn_ram = result['P_RAM_1/T_N']
    one_over_tn_raj = result['P_RAJ_1/T_N']
    
    return n_ram, n_raj, p_ram, p_raj, one_over_tn_ram, one_over_tn_raj
    

#### Compute woehler curve

In [ ]:
#%%script false --no-raise-error

load_list = np.exp(np.linspace(np.log(100), np.log(5e3), 10))
print(f"load values: {load_list}")

df_result = pd.DataFrame(index=pd.Index(load_list, name="load"))

# loop over different load values
for load in load_list:
    
    # perform the FKM nonlinear assessment
    n_ram, n_raj, p_ram, p_raj, one_over_tn_ram, one_over_tn_raj = get_lifetime(load)

    # store the results
    df_result.loc[load, "n_ram"] = n_ram
    df_result.loc[load, "p_ram"] = p_ram
    df_result.loc[load, "one_over_tn_ram"] = one_over_tn_ram
    df_result.loc[load, "n_raj"] = n_raj
    df_result.loc[load, "p_raj"] = p_raj
    df_result.loc[load, "one_over_tn_raj"] = one_over_tn_raj

In [ ]:
df_result

In [ ]:
plt.figure(figsize=(10,8))
plt.rcParams.update({'font.size': 22})

# woehler curve for P_RAM
p = plt.plot(df_result.n_ram, df_result.index, 'o-', label="RAM")
plt.plot(df_result.n_ram / np.sqrt(df_result.one_over_tn_ram), df_result.index, ':', color=p[0].get_color())
plt.plot(df_result.n_ram * np.sqrt(df_result.one_over_tn_ram), df_result.index, ':', color=p[0].get_color())

# woehler curve for P_RAJ
p = plt.plot(df_result.n_raj, df_result.index, 'o-', label="RAJ")
plt.plot(df_result.n_raj / np.sqrt(df_result.one_over_tn_raj), df_result.index, ':', color=p[0].get_color())
plt.plot(df_result.n_raj * np.sqrt(df_result.one_over_tn_raj), df_result.index, ':', color=p[0].get_color())

plt.grid()
plt.legend()
plt.loglog()
plt.xlabel("Lifetime N [-]")
plt.ylabel("Load L_o [N]")